# Minimal working example using BERT (`distBERT` model)

In [ ]:
# !pip install pandas torch transformers faiss-cpu

## Setup

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModel
import faiss

## Pseudo data creation

In [ ]:
# ----------------------------
# 1) Sample pseudo "data"
# ----------------------------
items = pd.DataFrame([
    {"item_id":"i1","title":"Noise Cancelling Headphones","description":"Wireless noise-cancelling headphones with 30-hour battery life","category":"electronics"},
    {"item_id":"i2","title":"Mechanical Keyboard","description":"Mechanical keyboard with RGB and hot-swappable switches","category":"electronics"},
    {"item_id":"i3","title":"Running Shoes","description":"Running shoes designed for long distance comfort and stability","category":"sports"},
    {"item_id":"i4","title":"Vegetarian Cookbook","description":"Cookbook featuring quick vegetarian recipes for busy weeknights","category":"books"},
    {"item_id":"i5","title":"Fitness Smartwatch","description":"Smartwatch with heart rate monitoring, GPS, and sleep tracking","category":"electronics"},
])

events = pd.DataFrame([
    {"user_id":"u1","item_id":"i1","ts":"2026-01-10"},
    {"user_id":"u1","item_id":"i5","ts":"2026-01-11"},
    {"user_id":"u2","item_id":"i2","ts":"2026-01-12"},
    {"user_id":"u3","item_id":"i3","ts":"2026-01-13"},
    {"user_id":"u3","item_id":"i4","ts":"2026-01-14"},
])
events["ts"] = pd.to_datetime(events["ts"])


## BERT encoder (What makes BERT work?)

In [ ]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
MODEL_NAME = "distilbert-base-uncased"  # for illustration, 66M model

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
encoder = AutoModel.from_pretrained(MODEL_NAME).to(DEVICE)


**Evaluate the BERT encoder**

In [ ]:
# ----------------------------
# 2) BERT encode helper
# ----------------------------

# turn off gradient calculation (no back propagation in using BERT)
@torch.no_grad()
def bert_embed(texts, max_len=128):
    batch = tokenizer(
        texts, padding=True, truncation=True, max_length=max_len, return_tensors="pt"
    )
    batch = {k: v.to(DEVICE) for k, v in batch.items()}
    out = encoder(**batch)
    cls = out.last_hidden_state[:, 0]          # first column [CLS]-like token for classification
    emb = F.normalize(cls, dim=-1)             # normalization
    return emb.cpu().numpy().astype("float32") # size (B, 768)


## Embedding items into vectors (for comparisons)

In [ ]:
# ----------------------------
# 3) Offline job: item embeddings
# ----------------------------

items["text"] = items["title"] + ". " + items["description"]
item_vecs = bert_embed(items["text"].tolist())
item_id_list = items["item_id"].tolist()

# Build ANN index (inner product works with normalized vectors)
index = faiss.IndexFlatIP(item_vecs.shape[1])
index.add(item_vecs)


## What user-specific data are there?

In [ ]:
# ----------------------------
# 4) Feature builder: user text from last N clicks
# ----------------------------
def build_user_text(user_id, events, items, N=3):
    hist = (events[events["user_id"] == user_id]
            .sort_values("ts")
            .tail(N)["item_id"]
            .tolist())
    if not hist:
        return "no history"
    text = items.set_index("item_id").loc[hist, "text"].tolist()
    return " ".join(text), set(hist)


## How to recommend "similar" item with BERT?

In [ ]:
# ----------------------------
# 5) "What to recommend" function
# ----------------------------
def recommend(user_id, k=3):
    user_text, seen = build_user_text(user_id, events, items, n=3)
    u = bert_embed([user_text])  # (1, 768)
    scores, idx = index.search(u, k + len(seen))  # keep track of what was seen by the user
    recs = []
    for j in idx[0]:
        iid = item_id_list[j]
        if iid not in seen:
            recs.append(iid)
        if len(recs) == k:
            break
    return recs


In [ ]:

for u in ["u1","u2","u3"]:
    print(u, "->", recommend(u, k=3))
